In [10]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
from scipy.misc import imread
import imagebase
import neighbour

def to_1d(rgb_image):
    ret = []
    x_dim, y_dim, z = rgb_image.shape
    for i in range(0, x_dim):
        ret_x = []
        for j in range(0, y_dim):
            ret_x.append(round(np.sum(rgb_image[i,j])/3, 1))
        ret.append(ret_x)
    return np.array(ret)

def denoise(image):
    x = image.shape[0]
    y = image.shape[1]
    tempimage = image
    for t in range(5):
        for i in range(x):
            for j in range(y):
                w = 0
                for ne in neighbour.neighbours(i,j,x,y,4):
                    w += ((2*image[ne]-1)*(2*image[i,j]-1))
                
                tempimage[i,j] = (w*image[i,j]+1)/2
        image = tempimage
    return im

prop = 0.7
varSigma = 0.1

im = to_1d(imageio.imread('pug.jpg'))
im = im/255


fig = plt.figure()
ax = fig.add_subplot(231)
ax.imshow(im,cmap='gray')
imG = imagebase.add_gaussian_noise(im,prop,varSigma)
ax2 = fig.add_subplot(232)
ax2.imshow(imG,cmap='gray')
imS = imagebase.add_saltnpeppar_noise(im,prop)
ax3 = fig.add_subplot(233)
ax3.imshow(imS,cmap='gray')
print("pre denG")
denG = denoise(imG)
print("pre denS")
denS = denoise(imS)
ax4 = fig.add_subplot(235)
ax4.imshow(denG,cmap='gray')
ax5 = fig.add_subplot(236)
ax5.imshow(denS,cmap='gray')

plt.savefig("icm.png")

pre denG


C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in double_scalars
C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in double_scalars


pre denS


In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
from scipy.misc import imread
import imagebase
import neighbour
import math


def to_1d(rgb_image):
    ret = []
    x_dim, y_dim, z = rgb_image.shape
    for i in range(0, x_dim):
        ret_x = []
        for j in range(0, y_dim):
            ret_x.append(round(np.sum(rgb_image[i,j])/3, 1))
        ret.append(ret_x)
    return np.array(ret)

def sigm(v):
    return (1 / (1 + np.exp(-v)))

#takes an image point value a 0..1
#takes a check value v 0..1
#returns Likelihood that a yields v
def p(a,v):
    res = 0
    thresh = v /2
    if(a >= thresh):
        res = 1
    else:
        res = 0
    return res

def newp(a,v):
    return (a + (v-a)/2)
    
def strongp(a,v):
    return (a + 3*(v-a)/4)

def l(image,i,j,x,y,v):
    res = 0
    sum = 0
    for ne in neighbour.neighbours(i,j,x,y,4):
        sum += p(image[ne],image[i][j])
    swap = sum/8
    if(image[i][j] == v):
        res = 1 - swap
    else:
        res = 0 + swap
    return res

def mfvb(image,iter):

    x = image.shape[0]
    y = image.shape[1]
    #Initialise the variational distributions
    #Initialise M
    m = np.zeros((iter+1,x,y),order='C')
    mu = np.ones((iter+1,x,y),order='C')
    
    q = np.zeros((x,y),order='C')
    #image is latent

    #correct access is m[t,x,y]
    for t in range(iter):
        #[ii][jj] pair comprises the singular i iterator used in CW specification
        print(t)
        for ii in range(x):
            for jj in range(y):
                sum = 0
                for ne in neighbour.neighbours(ii,jj,x,y,8):
                    #Add the -1..1 product value of neighbour with current, modified by the mu value for neighbour
                    sum += (2*image[ne] -1) * (2*image[ii][jj] -1) * mu[t,ne[0],ne[1]]

                m[t+1,ii,jj] = sum
                #mu = tanh(m_i^tau + 1/2(L_i(1) - L_i(-1)))
                mu[t+1,ii,jj] = math.tanh(m[t,ii,jj] + 1/2*(newp(image[ii][jj],1) - newp(image[ii][jj],0)))

    qProb = 1
    for i in range(x):
        for j in range(y):
            q[i,j] = sigm(2*(m[iter,i,j] + 1/2*(newp(image[i][j],1) - newp(image[i][j],0))))
            qProb *= q[i,j]

    #returns variational distro and mu
    return (q,mu,m)


prop = 0.7
varSigma = 0.1

im = to_1d(imageio.imread('pug.jpg'))
#im[i] = [0..1]
im = im/255
x = im.shape[0]
y = im.shape[1]

fig = plt.figure()
ax = fig.add_subplot(231)
ax.imshow(im,cmap='gray')
imG = imagebase.add_gaussian_noise(im,prop,varSigma)
ax2 = fig.add_subplot(232)
ax2.imshow(imG,cmap='gray')
imS = imagebase.add_saltnpeppar_noise(im,prop)
ax3 = fig.add_subplot(233)
ax3.imshow(imS,cmap='gray')

qS,muS,mS = mfvb(imS,7)
qG,muG,mG = mfvb(imG,7)


for i in range(x):
    for j in range(y):
        if(qS[i,j] < 0.5):
            if(imS[i,j] >= 0.5):
                imS[i,j] = strongp(imS[i,j],0)
            else:
                imS[i,j] = strongp(imS[i,j],1)
        else:
            if(imS[i,j] >= 0.5):
                imS[i,j] = strongp(imS[i,j],1)
            else:
                imS[i,j] = strongp(imS[i,j],0)

for i in range(x):
    for j in range(y):
        if(qG[i,j] < 0.5):
            if(imG[i,j] >= 0.5):
                imG[i,j] = strongp(imG[i,j],0)
            else:
                imG[i,j] = strongp(imG[i,j],1)
        else:
            if(imG[i,j] >= 0.5):
                imG[i,j] = strongp(imG[i,j],1)
            else:
                imG[i,j] = strongp(imG[i,j],0)


ax4 = fig.add_subplot(235)
ax4.imshow(imG,cmap='gray')
ax5 = fig.add_subplot(236)
ax5.imshow(imS,cmap='gray')

plt.savefig("vb.png")

0
1
2
3
4
5
6
0
1
2
3


In [3]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
from scipy.misc import imread
import imagebase
import neighbour
import math

im = to_1d(imageio.imread('pug.jpg'))
#im[i] = [0..1]
im = im/255

iter = 2

x = im.shape[0]
print(x)
y = im.shape[1]
print(y)
mu = np.ones((iter+1,x,y),order='C')
print(mu.shape)


for t in range(iter):
    for j in range(y):
        for i in range(x):
            mu[t,i,j] = t*10 + j + i
print(mu)






540
683
(3, 540, 683)
[[[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   6.80000000e+02
     6.81000000e+02   6.82000000e+02]
  [  1.00000000e+00   2.00000000e+00   3.00000000e+00 ...,   6.81000000e+02
     6.82000000e+02   6.83000000e+02]
  [  2.00000000e+00   3.00000000e+00   4.00000000e+00 ...,   6.82000000e+02
     6.83000000e+02   6.84000000e+02]
  ..., 
  [  5.37000000e+02   5.38000000e+02   5.39000000e+02 ...,   1.21700000e+03
     1.21800000e+03   1.21900000e+03]
  [  5.38000000e+02   5.39000000e+02   5.40000000e+02 ...,   1.21800000e+03
     1.21900000e+03   1.22000000e+03]
  [  5.39000000e+02   5.40000000e+02   5.41000000e+02 ...,   1.21900000e+03
     1.22000000e+03   1.22100000e+03]]

 [[  1.00000000e+01   1.10000000e+01   1.20000000e+01 ...,   6.90000000e+02
     6.91000000e+02   6.92000000e+02]
  [  1.10000000e+01   1.20000000e+01   1.30000000e+01 ...,   6.91000000e+02
     6.92000000e+02   6.93000000e+02]
  [  1.20000000e+01   1.30000000e+01   1.40000000e+01 ..

In [4]:
alpha = np.zeros((2,2,2))
for x in range(2):
    for y in range(2):
        alpha[x,y:] = x+(y*10)
print(alpha)
print(alpha[0,0,0])

#correct access is m[iteration,pixel,variable]

[[[  0.   0.]
  [ 10.  10.]]

 [[  1.   1.]
  [ 11.  11.]]]
0.0


In [5]:
beta = np.zeros((2,4,4))

for x in range(4):
    for y in range(4):
        for z in range(2):
            beta[z,y,x] = z + 10*x + 100*y
print(beta)

[[[   0.   10.   20.   30.]
  [ 100.  110.  120.  130.]
  [ 200.  210.  220.  230.]
  [ 300.  310.  320.  330.]]

 [[   1.   11.   21.   31.]
  [ 101.  111.  121.  131.]
  [ 201.  211.  221.  231.]
  [ 301.  311.  321.  331.]]]
